In [56]:
from typing import List

import networkx as nx
from networkx import diameter, is_connected
from tqdm import tqdm

from rumor_centrality.evaluation import hop_distances
from rumor_centrality.graph_clustering import assign_all_nodes_cluster, visualise_cluster_graph, \
    multiple_rumor_source_prediction


from rumor_centrality.graph_generator import small_world
from rumor_centrality.graph_simulations import si

In [9]:
graph = small_world(200, 6, 0.4)
some_graph_simulated, sources = si(graph.copy(), 10, 0.4, 2)
cluster_assgm = assign_all_nodes_cluster(some_graph_simulated, sources)
ext_sources = [[s] for s in sources]

100%|██████████| 10/10 [00:00<00:00, 2242.70it/s]


In [10]:
from functools import partial

node_one_set = [node for node, cluster in enumerate(cluster_assgm) if cluster == 0]

visualise_cluster_graph(some_graph_simulated, rumor_centers=ext_sources, assignment=cluster_assgm,
                        layout=partial(nx.spectral_layout, center=sources))

In [16]:
def flatten_list(l: List[List[int]]) -> List[int]:
    return [sl[0] for sl in l]

In [17]:
subgraphs_rumor_centers, assignm = multiple_rumor_source_prediction(graph, 2)

hop_distances(graph, flatten_list(subgraphs_rumor_centers), sources)

[3, 4]

In [60]:
max_infected_nodes = [100]#, 1000]
node_count = 3000
infection_prob = 0.3
num_infection_centers = [2, 3, 4, 5, 7, 10]
exp_iterations = 10

In [64]:
def multiple_sources_experiment(num_infection_centers, infection_prob, node_count, max_infected_nodes, graph_callback):
    exp_graph = graph_callback(node_count)

    while True:
        exp_graph_simulated, infection_sources = si(exp_graph.copy(), iterations=-1,
                                                infections_centers=num_infection_centers,
                                                max_infected_nodes=max_infected_nodes, infection_prob=infection_prob)
        if is_connected(exp_graph_simulated):
            break

    exp_diameter = diameter(exp_graph_simulated)
    subgraphs_rumor_centers, assignm = multiple_rumor_source_prediction(exp_graph_simulated, 2)

    hops = hop_distances(exp_graph, flatten_list(subgraphs_rumor_centers), infection_sources)

    return {
        "hops": hops,
        "graph_normalized_hops": list(map(lambda x: x / exp_diameter, hops)),
        "diameter": exp_diameter,
    }

In [65]:
multiple_sources_experiment(2, 0.3, 2000, 1000, partial(small_world, k=5, p=0.2))

{'hops': [7, 7],
 'graph_normalized_hops': [0.3888888888888889, 0.3888888888888889],
 'diameter': 18}

In [66]:
def get_small_world_graphs(ks=None, ps=None):
    if ps is None:
        ps = [0.1, 0.2, 0.4, 0.6]
    if ks is None:
        ks = [2, 3, 4, 5, 7, 9]
    for k in ks:
        for p in ps:
            yield (k, p), partial(small_world, k=k, p=p)

In [ ]:
result_small_world = {}

for num_infection_center in tqdm(num_infection_centers):
    result_small_world[num_infection_center] = {}

    for max_inf_nodes in max_infected_nodes:
        result_small_world[num_infection_center][max_inf_nodes] = {}

        for graph_params, graph in get_small_world_graphs():
            result_small_world[num_infection_center][max_inf_nodes][graph_params] = []

            for iteration in range(exp_iterations):
                result_small_world[num_infection_center][max_inf_nodes][graph_params].append(
                    multiple_sources_experiment(num_infection_center, infection_prob, node_count, max_inf_nodes, graph)
                )

  0%|          | 0/6 [00:00<?, ?it/s]